In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from glob import glob
from pathlib import Path

In [3]:
import pandas as pd

In [4]:
participants = [Path(path).stem for path in glob('./data/wesad/**') if Path(path).is_dir()]

In [5]:
for participant in participants:
    data = pd.read_pickle(f'./data/wesad/{participant}/{participant}.pkl')
    df = pd.DataFrame({
        'signal': data['signal']['chest']['ECG'].flatten(),
        'label': data['label']
    })
    df.to_csv(f'./data/wesad/{participant}/{participant}.csv')

In [6]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import collections
from datasets import load_dataset
import neurokit2 as nk

In [7]:
def TINN(x:np.array):
  """ Compute all the triangular interpolation to calculate the TINN scores. It also computes HRV index from an array x which contains 
      all the interbeats times for a given ECG signal.

      The axis is divided in 2 parts respectively on the right and left of the abscissa of the maximum value of the gaussian distribution
      The TINN score calculation is defined in the WESAD Dataset paper, to calculate it we needthe closest triangular interpolation 
      of the gaussian distribution of the interbeats times. The triangular interpolation is defined by 2 lines that meet at the maximum value
      of the gaussian distribution and cross the x-axis in N on the first half of the x-axis and M on the second half of the x-axis. 
      Thus inside ]N;M[ the interpolation function != 0
      Outside of ]N;M[ the interpolation function equals 0.
  """

  kernel = stats.gaussian_kde(x) #Create an approximated kernel for gaussian distribution from the x array (interbeats times)
  absi=np.linspace(np.min(x),np.max(x),len(x)) # Compute the x-axis of the interbeats distribution (from minimum interbeat time to maximum interbeat time)
  val=kernel.evaluate(absi) # Fit the gaussian distribution to the created x-axis
  ecart=absi[1]-absi[0] # Space between 2 values on the axis
  maxind=np.argmax(val) # Select the index for which the gaussian distribution (val array) is maximum 
  max_pos=absi[maxind]  # Interbeat time (abscissa) for which the gaussian distribution is maximum
  maxvalue=np.amax(val) # Max of the gaussian distribution
  N_abs=absi[0:maxind+1] # First half of the x-axis
  M_abs=absi[maxind:] # Second half of the x-axis
  HRVindex=len(x)/maxvalue
  err_N=[]
  err_M=[]

  for i in range(0,len(N_abs)-1):
    N=N_abs[i]
    slope=(maxvalue)/(max_pos-N)
    D=val[0:maxind+1]
    q=np.clip(slope*ecart*np.arange(-i,-i+maxind+1),0,None) #Triangular interpolation on the First half of the x-axis
    diff=D-q 
    err=np.multiply(diff,diff)
    err1=np.delete(err,-1)
    err2=np.delete(err, 0)
    errint=(err1+err2)/2
    errtot=np.linalg.norm(errint) # Error area between the triangular interpolation and the gaussian distribution on the first half of the x-axis
    err_N.append((errtot,N,N_abs,q))
  
  for i in range(1,len(M_abs)):
    M=M_abs[i]
    slope=(maxvalue)/(max_pos-M)
    D=val[maxind:]
    q=np.clip(slope*ecart*np.arange(-i,len(D)-i),0,None) #Triangular interpolation on the second half of the x-axis
    diff=D-q
    err=np.multiply(diff,diff)
    err1=np.delete(err,-1)
    err2=np.delete(err, 0)
    errint=(err1+err2)/2
    errtot=np.linalg.norm(errint) # Error area between the triangular interpolation and the gaussian distribution on the second half of the x-axis
    err_M.append((errtot,M,M_abs,q))

  return (err_N,err_M,absi,val,HRVindex)

def best_TINN(x:np.array):
  """Select the best N and M that give the best triangular interpolation function approximation of the gaussian distrbution and return
    N; M; the TINN score = M-N ; and the HRV index
  
  """
  err_N,err_M,_,_,HRVindex=TINN(x)
  N=np.argmin(np.array(err_N,dtype=object)[:,0])
  M=np.argmin(np.array(err_M,dtype=object)[:,0])
  absN=err_N[N][1]
  absM=err_M[M][1]
  return float(absN),float(absM),float(absM-absN),HRVindex

In [8]:
def get_freq_features_ecg(x):
  """ Returns frequential features of the Heart Rate Variability signal (interbeats times) by computing FFT, to compute the Fouriers 
  Frequencies the mean of the Heart Rate variability is used as sampling period  
  """
  mean=np.mean(x)
  yf=np.array(scipy.fft.fft(x-mean))
  xf=scipy.fft.fftfreq(len(x),mean)[0:len(x)//2]
  psd=(2/len(yf))*np.abs(yf)[0:len(x)//2]
  fmean=np.mean(xf)
  fstd=np.std(xf)
  sumpsd=np.sum(psd)
  return fmean,fstd,sumpsd


In [9]:
def num_compare_NN50(x,i):
  """Count the number of HRV intervals differing more than 50 ms for a given HRV interval x[i]
  
  """
  ref=x[i]
  k=0
  diff=np.absolute(x-ref)
  k+=np.sum(np.where(diff>0.05,1,0))
  return k 

def compare_NN50(x):
  """ Returns the number and percentage of HRV intervals differing more than 50ms for all intervals
  
  """
  k=0
  for i in range(0,len(x)):
    k+=num_compare_NN50(x,i)
  if k==0:
    k=1
  return k,(k/(len(x)*len(x)))

In [10]:
sampling_rate = 700
n_window = 60 * sampling_rate

In [11]:
window_shift_size = 0.25
step_size = int(window_shift_size * sampling_rate)

In [12]:
from tqdm import tqdm

In [13]:
from scipy.signal import find_peaks
from scipy.fft import fft, fftfreq

In [48]:

def f_fr_n(freq, max_freq, l ):
    if freq < max_freq:
        return int(freq * l/max_freq)
    else:
        return l - 1
    
def detect_peaks_ECG(sample, window_size,timestep_data,distance):
    f_p = find_peaks(sample, height = 0.4, distance = distance)
    #time features
    f_p_diff = np.diff(f_p[0]) * timestep_data
    
    # heart rate mean std min max 
    HR_mean = (60/f_p_diff).mean()
    HR_std = (60/f_p_diff).std()
    HR_max = (60/f_p_diff).max()
    HR_min = (60/f_p_diff).max()
    #NN50
    #pNN50
    NN50 = sum(np.abs(np.diff(f_p_diff)) > 0.050)
    N_HRV_50 = NN50
    P_HRV_50 = NN50/len(f_p_diff)
    #rr_features
    rmssd = np.sqrt(np.mean(np.square(np.diff(f_p_diff))))
    rr_mean = f_p_diff.mean()
    rr_std = f_p_diff.std()
    # freq features
    # f_p_diff_fft = savgol_filter(np.diff(f_p_diff), 5,2)
    
    T = window_size * timestep_data
    k = np.arange(len(f_p_diff))
    freqs = k/T
    m = freqs.max()/2
    l = int(len(freqs)/2)
    ffts = abs(np.fft.fft(f_p_diff)*np.hamming(len(k)))**2
    ULF = sum( ffts[ f_fr_n(0.01,m,l):f_fr_n(0.04,m,l) ] )
    HF = sum( ffts[ f_fr_n(0.15,m,l):f_fr_n(0.4,m,l) ] )
    LF = sum( ffts[ f_fr_n(0.04,m,l):f_fr_n(0.15,m,l) ] )
    UHF = sum( ffts[ f_fr_n(0.4,m,l):f_fr_n(1,m,l) ] )
    
    TP = ULF + LF + HF + UHF

    rate_L_H = LF/HF
    lfN = LF / TP 
    hfN = HF / TP
    
    return {
        'μhr' : HR_mean,
        'σhr' : HR_std,
        # 'HR_max': HR_max,
        # 'HR_min' : HR_min,
        'NN50' : N_HRV_50,
        'pNN50' : P_HRV_50,
        # 'rmssd' : rmssd,
        # 'rr_mean' : rr_mean,
        # 'rr_std' : rr_std,
        'ULF' : ULF,
        'HF': HF,
        'LF': LF,
        'UHF': UHF,
        'LF_HF_ratio': rate_L_H,
        'Σ': TP,
        'relative_power_ULF': (ULF / TP) * 100,
        'relative_power_LF': (LF / TP) * 100,
        'relative_power_HF': (HF / TP) * 100,
        'relative_power_UHF': (UHF / TP) * 100,
        'LF_norm': lfN,
        'HF_norm': hfN,
    }

In [53]:
def process_and_save_participant(participant):
    dataset = load_dataset('csv', data_files=f'./data/wesad/{participant}/{participant}.csv')['train']

    dataframes = []
    with tqdm(total=len(dataset)) as pbar:
        for start_idx in range(0, len(dataset), step_size): ## Window shift
            sample = dataset[start_idx:start_idx+n_window]
            if len(sample['signal']) < n_window:
                continue
            
            signal = sample['signal']
            label = collections.Counter(sample['label']).most_common(1)[0][0]

            peaks, _ = nk.ecg_peaks(signal, sampling_rate=sampling_rate)
            peaks_indices = peaks[peaks['ECG_R_Peaks'] == 1].index

            ## HR
            # signal_rate = nk.signal_rate(peaks, sampling_rate=sampling_rate)
            # mean_hr = np.mean(signal_rate)
            # std_hr = np.std(signal_rate)

            ## Periods
            # print("Getting Periods...")
            # periods = np.array([(peaks_indices[i+1]-peaks_indices[i])/sampling_rate for i in range(0,len(peaks_indices)-1)])
            # frequency = 1 / periods
            # mean_freq = np.mean(frequency)
            # std_freq = np.std(frequency)
            # mean_f, std_f, sum_psd = get_freq_features_ecg(periods)
            
            ## HRV
            hrv = np.array([(peaks_indices[i]-peaks_indices[i-1])/sampling_rate for i in range(1,len(peaks_indices))])
            mean_hrv = np.mean(hrv)
            std_hrv = np.std(hrv)
            rms_hrv = np.sqrt(np.mean(hrv**2))
            # _, _, _, hrv_index = best_TINN(hrv)

            ## %NN50
            # NN50, pNN50 = compare_NN50(hrv)
            
            # rr_intervals = np.diff(find_peaks(sample['signal'])[0])
            # rr_fft = fft(rr_intervals)  
            # n_samples = len(rr_intervals)
            # rr_frequencies = fftfreq(n_samples, d=1/sampling_rate) 
            # # ULF band isolation
            # ulf_band_mask = (rr_frequencies >= 0.01) & (rr_frequencies <= 0.04) 
            # lf_band_mask = (rr_frequencies >= 0.04) & (rr_frequencies <= 0.15) 
            # hf_band_mask = (rr_frequencies >= 0.15) & (rr_frequencies <= 0.4) 
            # uhf_band_mask = (rr_frequencies >= 0.4) & (rr_frequencies <= 1) 
            # ulf_fft = rr_fft[ulf_band_mask]
            # lf_fft = rr_fft[lf_band_mask]
            # hf_fft = rr_fft[hf_band_mask]
            # uhf_fft = rr_fft[uhf_band_mask]

            # power calculation
            # ulf = np.sum(np.abs(ulf_fft)**2)
            # lf = np.sum(np.abs(lf_fft)**2)
            # hf = np.sum(np.abs(hf_fft)**2)
            # uhf = np.sum(np.abs(uhf_fft)**2)

            # ulf = nk.signal_power(sample['signal'], [0.01, 0.04], sampling_rate=sampling_rate)['Hz_0.01_0.04']
            # lf = nk.signal_power(sample['signal'], [0.04, 0.15], sampling_rate=sampling_rate)['Hz_0.04_0.15']
            # hf = nk.signal_power(sample['signal'], [0.15, 0.4], sampling_rate=sampling_rate)['Hz_0.15_0.4']
            # uhf = nk.signal_power(sample['signal'], [0.4, 1], sampling_rate=sampling_rate)['Hz_0.4_1']
            # tp = ulf + lf + hf + uhf
            # lfhf = lf/hf
            # lfN = lf / tp 
            # hfN = hf / tp

            ## Dataframe
            hrv = nk.hrv(peaks, sampling_rate=sampling_rate)

            fp_data = detect_peaks_ECG(sample['signal'], n_window, 1/sampling_rate, 200)

            df = pd.DataFrame({
                'label': label,
                'μhr': fp_data['μhr'],
                'σhr': fp_data['σhr'],
                'μhrv': mean_hrv,
                'σhrv': std_hrv,
                'NN50': fp_data['NN50'],
                'pNN50': fp_data['pNN50'],
                'TINN': hrv['HRV_TINN'],
                'rmsHRV': rms_hrv,
                'ULF': fp_data['ULF'],
                'LF': fp_data['LF'],
                'HF': fp_data['HF'],
                'UHF': fp_data['UHF'],
                'LF_HF_ratio': fp_data['LF_HF_ratio'],
                'Σ': fp_data['Σ'],
                'relative_power_ulf': fp_data['relative_power_ULF'],
                'relative_power_lf': fp_data['relative_power_LF'],
                'relative_power_hf': fp_data['relative_power_HF'],
                'relative_power_uhf': fp_data['relative_power_UHF'],
                'LF_norm': fp_data['HF_norm'],
                'HF_norm': fp_data['HF_norm'],
            })

            dataframes.append(df)
            tqdm.update(pbar, step_size)
        
    result = pd.concat(dataframes, ignore_index=True)
    result.to_csv(f'./data/wesad_features_60s/{participant}.csv')

In [54]:
from joblib import Parallel, delayed

In [55]:
Parallel(n_jobs=4)(delayed(process_and_save_participant)(participant) for participant in participants) 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]